# Unbiased Pronoun Resolution

## Setup

### Imports

In [1]:
from src.utils import *
from src.data_utils import *
from src.models import *
from src.train import *
from src.zs_utils import *
import os

%load_ext autoreload
%autoreload 2

/opt/conda/envs/dl4nlp-pr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Set config

In [5]:
# set_seed(42)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f'using device: {device}')

## Task: Zero-shot CR using LLMs

### GAP

In [ ]:
# classify gap test set zs using gpt3
data_path = 'data/gap/gap-test.tsv'
num_samples = 10
output_path = None

# call gpt api
responses = classify_coref_zs(data_path, num_samples, output_path=output_path)
for i, response in enumerate(responses):
    print(f'response {i}: {response}')

## Fast Coref

### Zero-shot eval on GAP

In [6]:
# classify gap test set zs using fcoref
data_path = 'data/gap/gap-test.tsv'  # path to gap test set
num_samples = None # number of samples to classify
model_name = 'lmcoref'  # either 'fcoref' or 'lmcoref'
# init fcoref classifier
fcr_classifier = FCorefClassifier(model_name, data_path, num_samples)
# get cluster preds
cluster_preds = fcr_classifier.pred_cr_clusters()
# get label preds
labels, preds = fcr_classifier.pred_cr_labels(verbose=False)

# eval 
metrics = ['acc', 'f1'] # metrics to eval
res_file_path = None # path to save results
print(f'evaluating metrics on {num_samples} for {model_name} on GAP test...')
eval_res = eval_preds(labels, preds, metrics)
for metric, val in eval_res.items():
    print(f'{metric}: {val:.3f}')

10/12/2023 14:50:44 - INFO - 	 missing_keys: []
10/12/2023 14:50:44 - INFO - 	 unexpected_keys: []
10/12/2023 14:50:44 - INFO - 	 mismatched_keys: []
10/12/2023 14:50:44 - INFO - 	 error_msgs: []
10/12/2023 14:50:44 - INFO - 	 Model Parameters: 590.0M, Transformer: 434.6M, Coref head: 155.4M
10/12/2023 14:50:44 - INFO - 	 Tokenize 2000 inputs...
Map: 100%|██████████| 2000/2000 [00:07<00:00, 270.48 examples/s]
10/12/2023 14:50:51 - INFO - 	 ***** Running Inference on 2000 texts *****
Inference: 100%|██████████| 2000/2000 [00:42<00:00, 47.53it/s]
10/12/2023 14:51:34 - INFO - 	 Tokenize 2000 inputs...
Map: 100%|██████████| 2000/2000 [00:06<00:00, 314.91 examples/s]
10/12/2023 14:51:40 - INFO - 	 ***** Running Inference on 2000 texts *****
Inference: 100%|██████████| 2000/2000 [00:41<00:00, 47.90it/s]


predicting labels for 2000 samples using LingMessCoref...


2000it [00:00, 13341.68it/s]

done.
evaluating metrics on None for lmcoref on GAP test...
acc: 0.707
f1: 0.674


## Modify GAP dataset to gender-neutral pronouns   

In [19]:
# create modified gap test set
gap_path = 'data/gap/gap-test.tsv'
gap_gn_path = 'data/gap/gap-test-gn-v2.tsv'
mod_texts = create_gn_gap(gap_path, gap_gn_path)

# sanity check for new offsets
# check_offsets(gap_gn_path, num_rows=1000)

0it [00:00, ?it/s]

/home/neil/dl4nlp-pr/src/data_utils.py:245: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'His' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_gap_gn.loc[i, 'Pronoun_old'] = row['Pronoun']    # save the old pronoun
5it [03:45, 45.05s/it]


ValueError: One or more markers not found in text.